In [1]:
!pip install pandas numpy requests statsmodels matplotlib schedule


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import schedule
import time
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime, timedelta


In [3]:
API_KEY = "7152afec4273f801d5f4015ceb174daf"
BASE_URL = "http://api.marketstack.com/v1/eod"

def get_stock_data(symbol, start_date, end_date):
    """Fetch real-time stock prices."""
    params = {
        "access_key": API_KEY,
        "symbols": symbol,
        "date_from": start_date,
        "date_to": end_date,
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    
    if "data" in data:
        df = pd.DataFrame(data["data"])
        df["date"] = pd.to_datetime(df["date"])
        df = df[["date", "close"]].rename(columns={"date": "Date", "close": "Close_Price"})
        df.sort_values("Date", inplace=True)
        return df
    else:
        print(f"Error fetching data for {symbol}: {data}")
        return pd.DataFrame()


In [4]:
def forecast_stock(symbol):
    """Train SARIMA model & forecast next 5 days."""
    print(f"🔄 Fetching live data for {symbol}...")
    end_date = datetime.today().strftime("%Y-%m-%d")
    start_date = (datetime.today() - timedelta(days=90)).strftime("%Y-%m-%d")
    df = get_stock_data(symbol, start_date, end_date)
    if df.empty:
        print(f"❌ No data found for {symbol}. Skipping forecast.")
        return
    df.set_index("Date", inplace=True)
    df = df.asfreq("D")  
    df = df.ffill() 
    model = SARIMAX(df["Close_Price"], order=(1,1,1), seasonal_order=(1,1,1,5))
    model_fit = model.fit()
    future_steps = 5
    forecast = model_fit.forecast(steps=future_steps)
    forecast_dates = pd.date_range(df.index[-1] + timedelta(days=1), periods=future_steps, freq="D")
    forecast_df = pd.DataFrame({"Date": forecast_dates, "Predicted_Price": forecast.values})
    print(f"✅ Forecast for {symbol}:")
    print(forecast_df)
    return forecast_df


In [5]:
STOCKS = ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN"]  
for stock in STOCKS:
    forecast_stock(stock)  


🔄 Fetching live data for AAPL...
✅ Forecast for AAPL:
                       Date  Predicted_Price
0 2025-03-19 00:00:00+00:00       213.633581
1 2025-03-20 00:00:00+00:00       213.203527
2 2025-03-21 00:00:00+00:00       213.159893
3 2025-03-22 00:00:00+00:00       212.298893
4 2025-03-23 00:00:00+00:00       211.568000
🔄 Fetching live data for GOOGL...
✅ Forecast for GOOGL:
                       Date  Predicted_Price
0 2025-03-19 00:00:00+00:00       160.807000
1 2025-03-20 00:00:00+00:00       159.873984
2 2025-03-21 00:00:00+00:00       160.366656
3 2025-03-22 00:00:00+00:00       159.931208
4 2025-03-23 00:00:00+00:00       158.268887
🔄 Fetching live data for MSFT...
✅ Forecast for MSFT:
                       Date  Predicted_Price
0 2025-03-19 00:00:00+00:00       383.760320
1 2025-03-20 00:00:00+00:00       382.700050
2 2025-03-21 00:00:00+00:00       380.999719
3 2025-03-22 00:00:00+00:00       380.364092
4 2025-03-23 00:00:00+00:00       379.901237
🔄 Fetching live data for T